In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import load_model
from importlib import reload
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation
from keras.optimizers import Adam
from PGAgent import PGAgent
import simulation_process


Using TensorFlow backend.


In [2]:
data=pd.read_csv("long_term_maximum_CPU_usage.csv")
data1=data.fillna(0)
data1=np.array(data1)
data1=data1[:1000,:]

In [3]:
row_num=data1.shape[0]
col_num=data1.shape[1]

In [1426]:
# Environment Settings
PM_num=20
# PM Trace Generation
vm_num=200
chosen_column=list(np.random.randint(col_num,size=vm_num))
PM_data=np.zeros([row_num,PM_num])
PM_data_state=np.zeros([row_num,PM_num])
column_front=0
column_end=0
PM_vm_list=[]
PM_vm_list1=[]
for i in range (PM_num):
    if i<PM_num/2:
        vm_num=15
    else:
        vm_num=5
    column_front=column_end
    column_end+=vm_num
    PM_task_list=list(chosen_column[column_front:column_end])
    PM_vm_list.append(PM_task_list)
    PM_vm_list1.append(PM_task_list.copy())
    for j in range (len(PM_task_list)):
        PM_data[:,i]+=data1[:,PM_task_list[j]]



In [1427]:
PM_data.shape
score=-20;

In [1791]:
# Environment Settings
PM_num=100;
# PM Trace Generation
PM_data=np.zeros([row_num,PM_num])
PM_vm_list=[]
PM_vm_list1=PM_vm_list.copy()
PM_vm_list2=PM_vm_list.copy()
for i in range (PM_num):
    chosen_vm_num=np.random.randint(20)
    chosen_column=list(np.random.randint(col_num,size=chosen_vm_num))
    #if i<PM_num/2:
    PM_vm_list.append(chosen_column)    
    PM_vm_list1.append(chosen_column.copy())
    PM_vm_list2.append(chosen_column.copy())
    for j in range( chosen_vm_num):
        PM_data[:,i]+=data1[:,chosen_column[j]]
   # else:
   #     PM_vm_list.append([])    
   #     PM_vm_list1.append([])
  #      PM_vm_list2.append([])

In [1792]:
reload(simulation_process)
state_size=PM_num
action_size=(PM_num-1)*3+1
agent=PGAgent(state_size,action_size)
#PM_intial_workload_data, PM_initial_workload=simulation_process.PM_current_workload(data1,PM_vm_list,15,PM_num)


Model: "sequential_120"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_354 (Dense)            (None, 50)                5050      
_________________________________________________________________
dense_355 (Dense)            (None, 298)               15198     
Total params: 20,248
Trainable params: 20,248
Non-trainable params: 0
_________________________________________________________________


C:\Users\zf7ja\Desktop\Prediction_Work1\src\PGAgent.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, input_shape=(100,), activation="relu", kernel_initializer="he_uniform")`
  model.add(Dense(100, input_shape=(self.state_size,),activation='relu', init='he_uniform'))


In [1793]:
reload(simulation_process)
current_time=0
PM_current_workload_data, PM_current_workload=simulation_process.PM_current_workload(data1,PM_vm_list,101+current_time,PM_num)
PM_current_workload_data1, PM_current_workload1=simulation_process.PM_current_workload(data1,PM_vm_list1,101+current_time,PM_num)
print(PM_current_workload1)
        
SLo_no_migration=len(list(filter(lambda x: x>0.95 , PM_current_workload1)))
state=simulation_process.observation_state(PM_current_workload,1)
agent.states, agent.probs, agent.gradients, agent.rewards = [], [], [], []

print("initial_state:",state)

[0.382874, 0.67995, 0.9481999999999999, 0.275909, 0.52722, 0.4844, 1.0865870000000002, 0.16953000000000001, 0.7207999999999999, 0.127942, 0, 1.6972900000000002, 0.533234, 1.1843250000000003, 1.0344399999999998, 0.45216999999999996, 0.30609000000000003, 0.272281, 1.4796899999999997, 0.7093799999999999, 1.3827950000000002, 0.8231229999999998, 1.4829300000000003, 0.6567299999999999, 0.576354, 0.0653959, 0.5784870000000001, 0.273011, 0.8816200000000001, 0.80027, 0.98894, 0.02441, 0.45803, 0.39261000000000007, 0.015007, 0.174493, 0.14667000000000002, 0.47219, 0.368147, 0.7195900000000002, 1.3709280000000001, 1.9799910000000003, 1.6880680000000001, 0.09677, 1.5400179999999997, 0.9386620000000001, 0.1775, 0.5692830000000001, 1.067604, 0.79567, 0.36987800000000004, 0.48460000000000003, 0.819827, 0.9017930000000002, 0.6471690000000001, 0.79858, 1.1845499999999998, 0.8635700000000001, 0.62392, 0.24932300000000002, 0, 0.9557990000000001, 0.62553, 1.10097, 0.10494, 0.7027789999999999, 0.02936, 1.1

In [1804]:
reload(simulation_process)
def RL_train_epoch(PM_num,state,PM_current_workload_data,PM_current_workload,PM_vm_list,PM_vm_list_status,train_status):
    agent.states, agent.probs, agent.gradients, agent.rewards = [], [], [], []
    mig_num=0
    for i in range(PM_num):
        NN_state=simulation_process.get_nn_state(state,i)
        if NN_state[0]==3:
            action,prob=agent.act(NN_state)
            state,reward,SLo=simulation_process.env_step(i,PM_current_workload_data,PM_current_workload,PM_vm_list,action,PM_vm_list_status)
            agent.remember(NN_state,action,prob,reward)
            mig_num+=1
   # print(agent.rewards)
    #print(agent.states)
    if(len(agent.rewards)>0 and train_status==1):
        agent.train()
    agent.states, agent.probs, agent.gradients, agent.rewards = [], [], [], []
    SLo=len(list(filter(lambda x: x>0.95 , PM_current_workload)))
    return SLo,state,mig_num

In [1795]:
import pdb; 
#pdb.set_trace()
SLo_no_sum=0
SLo_sum=0
for iter in range(5):
    SLo,state,_=RL_train_epoch(PM_num,state,PM_current_workload_data,PM_current_workload,PM_vm_list,0,1)

    print("after_state:",state)
    print("current time %d-----: SLO_no_migration:%d, SLO_RL:%d, " %(current_time,SLo_no_migration,SLo))

after_state: [1. 2. 2. 1. 2. 2. 2. 1. 2. 1. 1. 3. 2. 2. 2. 2. 1. 1. 3. 2. 3. 2. 3. 2.
 2. 1. 2. 1. 2. 2. 2. 2. 2. 1. 1. 1. 1. 2. 1. 2. 3. 3. 3. 1. 3. 2. 1. 2.
 3. 3. 1. 2. 2. 2. 2. 2. 3. 2. 2. 1. 1. 2. 2. 3. 1. 2. 1. 3. 2. 1. 1. 2.
 2. 1. 3. 2. 1. 2. 2. 2. 2. 2. 1. 3. 1. 2. 2. 3. 3. 1. 1. 2. 1. 1. 2. 3.
 1. 1. 1. 3.]
current time 0-----: SLO_no_migration:22, SLO_RL:17, 
after_state: [1. 2. 2. 2. 2. 2. 3. 1. 2. 1. 1. 3. 2. 2. 3. 2. 1. 1. 2. 2. 3. 2. 2. 2.
 2. 1. 2. 1. 2. 2. 2. 2. 2. 1. 1. 1. 1. 3. 1. 2. 2. 3. 3. 2. 3. 2. 1. 2.
 3. 3. 1. 2. 2. 2. 2. 2. 2. 2. 2. 1. 1. 2. 2. 3. 1. 2. 1. 3. 2. 1. 1. 2.
 2. 1. 2. 2. 2. 2. 2. 2. 2. 2. 1. 2. 1. 2. 2. 3. 2. 1. 1. 2. 1. 1. 2. 3.
 1. 1. 1. 3.]
current time 0-----: SLO_no_migration:22, SLO_RL:14, 
after_state: [1. 2. 3. 2. 2. 2. 2. 1. 2. 1. 1. 3. 2. 2. 3. 2. 2. 1. 2. 2. 3. 2. 2. 3.
 2. 1. 2. 1. 2. 2. 2. 2. 2. 1. 1. 1. 1. 3. 1. 2. 2. 3. 3. 2. 3. 2. 2. 2.
 2. 3. 1. 2. 2. 2. 2. 2. 2. 2. 2. 1. 1. 2. 2. 1. 1. 2. 1. 3. 2. 1. 1. 2.
 2. 1. 2. 2. 2. 2. 2. 

In [1796]:
agent.states

[]

In [1805]:
SLo_no_sum=0
SLo_sum=0
PM_vm_list=[]
for i in range(len(PM_vm_list1)):
    PM_vm_list.append(PM_vm_list1[i].copy())
    
for current_time in range(700):
    #re_score=score
    print ("current_time:",current_time)
    PM_current_workload_data1, PM_current_workload1=simulation_process.PM_current_workload(data1,PM_vm_list1,10+current_time,PM_num)
        #print(PM_current_workload1)    
    SLo_no_migration=len(list(filter(lambda x: x>0.95 , PM_current_workload1)))
    PM_current_workload_data, PM_current_workload=simulation_process.PM_current_workload(data1,PM_vm_list,10+current_time,PM_num)
    state=simulation_process.observation_state(PM_current_workload,1)
    #print("initial_state:",state)
    for epoch in range(1):
        SLo,state,mig_num=RL_train_epoch(PM_num,state,PM_current_workload_data,PM_current_workload,PM_vm_list,1,0)
   # print("after_state:",state)
    print("current time %d-----: SLO_no_migration:%d, SLO_RL:%d, " %(current_time,SLo_no_migration,SLo))
    SLo_sum+=SLo
    SLo_no_sum+=SLo_no_migration
    print("current time %d-----: SLO_no_sum:%d, SLO_sum:%d, " %(current_time,SLo_no_sum,SLo_sum))


current_time: 0
current time 0-----: SLO_no_migration:25, SLO_RL:15, 
current time 0-----: SLO_no_sum:25, SLO_sum:15, 
current_time: 1
current time 1-----: SLO_no_migration:19, SLO_RL:16, 
current time 1-----: SLO_no_sum:44, SLO_sum:31, 
current_time: 2
current time 2-----: SLO_no_migration:20, SLO_RL:7, 
current time 2-----: SLO_no_sum:64, SLO_sum:38, 
current_time: 3
current time 3-----: SLO_no_migration:23, SLO_RL:11, 
current time 3-----: SLO_no_sum:87, SLO_sum:49, 
current_time: 4
current time 4-----: SLO_no_migration:22, SLO_RL:4, 
current time 4-----: SLO_no_sum:109, SLO_sum:53, 
current_time: 5
current time 5-----: SLO_no_migration:23, SLO_RL:3, 
current time 5-----: SLO_no_sum:132, SLO_sum:56, 
current_time: 6
current time 6-----: SLO_no_migration:20, SLO_RL:1, 
current time 6-----: SLO_no_sum:152, SLO_sum:57, 
current_time: 7
current time 7-----: SLO_no_migration:23, SLO_RL:5, 
current time 7-----: SLO_no_sum:175, SLO_sum:62, 
current_time: 8
current time 8-----: SLO_no_migra

current time 71-----: SLO_no_migration:25, SLO_RL:6, 
current time 71-----: SLO_no_sum:1757, SLO_sum:545, 
current_time: 72
current time 72-----: SLO_no_migration:24, SLO_RL:9, 
current time 72-----: SLO_no_sum:1781, SLO_sum:554, 
current_time: 73
current time 73-----: SLO_no_migration:21, SLO_RL:4, 
current time 73-----: SLO_no_sum:1802, SLO_sum:558, 
current_time: 74
current time 74-----: SLO_no_migration:24, SLO_RL:3, 
current time 74-----: SLO_no_sum:1826, SLO_sum:561, 
current_time: 75
current time 75-----: SLO_no_migration:17, SLO_RL:6, 
current time 75-----: SLO_no_sum:1843, SLO_sum:567, 
current_time: 76
current time 76-----: SLO_no_migration:20, SLO_RL:5, 
current time 76-----: SLO_no_sum:1863, SLO_sum:572, 
current_time: 77
current time 77-----: SLO_no_migration:20, SLO_RL:6, 
current time 77-----: SLO_no_sum:1883, SLO_sum:578, 
current_time: 78
current time 78-----: SLO_no_migration:22, SLO_RL:6, 
current time 78-----: SLO_no_sum:1905, SLO_sum:584, 
current_time: 79
current 

current time 145-----: SLO_no_migration:25, SLO_RL:5, 
current time 145-----: SLO_no_sum:3550, SLO_sum:1064, 
current_time: 146
current time 146-----: SLO_no_migration:35, SLO_RL:11, 
current time 146-----: SLO_no_sum:3585, SLO_sum:1075, 
current_time: 147
current time 147-----: SLO_no_migration:31, SLO_RL:12, 
current time 147-----: SLO_no_sum:3616, SLO_sum:1087, 
current_time: 148
current time 148-----: SLO_no_migration:27, SLO_RL:13, 
current time 148-----: SLO_no_sum:3643, SLO_sum:1100, 
current_time: 149
current time 149-----: SLO_no_migration:27, SLO_RL:11, 
current time 149-----: SLO_no_sum:3670, SLO_sum:1111, 
current_time: 150
current time 150-----: SLO_no_migration:29, SLO_RL:9, 
current time 150-----: SLO_no_sum:3699, SLO_sum:1120, 
current_time: 151
current time 151-----: SLO_no_migration:23, SLO_RL:8, 
current time 151-----: SLO_no_sum:3722, SLO_sum:1128, 
current_time: 152
current time 152-----: SLO_no_migration:24, SLO_RL:7, 
current time 152-----: SLO_no_sum:3746, SLO_s

current time 214-----: SLO_no_migration:21, SLO_RL:9, 
current time 214-----: SLO_no_sum:5266, SLO_sum:1519, 
current_time: 215
current time 215-----: SLO_no_migration:26, SLO_RL:10, 
current time 215-----: SLO_no_sum:5292, SLO_sum:1529, 
current_time: 216
current time 216-----: SLO_no_migration:26, SLO_RL:10, 
current time 216-----: SLO_no_sum:5318, SLO_sum:1539, 
current_time: 217
current time 217-----: SLO_no_migration:29, SLO_RL:12, 
current time 217-----: SLO_no_sum:5347, SLO_sum:1551, 
current_time: 218
current time 218-----: SLO_no_migration:26, SLO_RL:9, 
current time 218-----: SLO_no_sum:5373, SLO_sum:1560, 
current_time: 219
current time 219-----: SLO_no_migration:24, SLO_RL:11, 
current time 219-----: SLO_no_sum:5397, SLO_sum:1571, 
current_time: 220
current time 220-----: SLO_no_migration:27, SLO_RL:11, 
current time 220-----: SLO_no_sum:5424, SLO_sum:1582, 
current_time: 221
current time 221-----: SLO_no_migration:27, SLO_RL:10, 
current time 221-----: SLO_no_sum:5451, SLO

current time 293-----: SLO_no_migration:18, SLO_RL:4, 
current time 293-----: SLO_no_sum:7095, SLO_sum:2008, 
current_time: 294
current time 294-----: SLO_no_migration:21, SLO_RL:5, 
current time 294-----: SLO_no_sum:7116, SLO_sum:2013, 
current_time: 295
current time 295-----: SLO_no_migration:18, SLO_RL:2, 
current time 295-----: SLO_no_sum:7134, SLO_sum:2015, 
current_time: 296
current time 296-----: SLO_no_migration:20, SLO_RL:5, 
current time 296-----: SLO_no_sum:7154, SLO_sum:2020, 
current_time: 297
current time 297-----: SLO_no_migration:21, SLO_RL:3, 
current time 297-----: SLO_no_sum:7175, SLO_sum:2023, 
current_time: 298
current time 298-----: SLO_no_migration:24, SLO_RL:7, 
current time 298-----: SLO_no_sum:7199, SLO_sum:2030, 
current_time: 299
current time 299-----: SLO_no_migration:22, SLO_RL:6, 
current time 299-----: SLO_no_sum:7221, SLO_sum:2036, 
current_time: 300
current time 300-----: SLO_no_migration:25, SLO_RL:7, 
current time 300-----: SLO_no_sum:7246, SLO_sum:2

current time 359-----: SLO_no_migration:22, SLO_RL:3, 
current time 359-----: SLO_no_sum:8385, SLO_sum:2283, 
current_time: 360
current time 360-----: SLO_no_migration:18, SLO_RL:2, 
current time 360-----: SLO_no_sum:8403, SLO_sum:2285, 
current_time: 361
current time 361-----: SLO_no_migration:15, SLO_RL:4, 
current time 361-----: SLO_no_sum:8418, SLO_sum:2289, 
current_time: 362
current time 362-----: SLO_no_migration:14, SLO_RL:0, 
current time 362-----: SLO_no_sum:8432, SLO_sum:2289, 
current_time: 363
current time 363-----: SLO_no_migration:21, SLO_RL:1, 
current time 363-----: SLO_no_sum:8453, SLO_sum:2290, 
current_time: 364
current time 364-----: SLO_no_migration:23, SLO_RL:3, 
current time 364-----: SLO_no_sum:8476, SLO_sum:2293, 
current_time: 365
current time 365-----: SLO_no_migration:18, SLO_RL:2, 
current time 365-----: SLO_no_sum:8494, SLO_sum:2295, 
current_time: 366
current time 366-----: SLO_no_migration:13, SLO_RL:1, 
current time 366-----: SLO_no_sum:8507, SLO_sum:2

current time 429-----: SLO_no_migration:14, SLO_RL:3, 
current time 429-----: SLO_no_sum:9586, SLO_sum:2467, 
current_time: 430
current time 430-----: SLO_no_migration:15, SLO_RL:5, 
current time 430-----: SLO_no_sum:9601, SLO_sum:2472, 
current_time: 431
current time 431-----: SLO_no_migration:18, SLO_RL:3, 
current time 431-----: SLO_no_sum:9619, SLO_sum:2475, 
current_time: 432
current time 432-----: SLO_no_migration:18, SLO_RL:4, 
current time 432-----: SLO_no_sum:9637, SLO_sum:2479, 
current_time: 433
current time 433-----: SLO_no_migration:19, SLO_RL:2, 
current time 433-----: SLO_no_sum:9656, SLO_sum:2481, 
current_time: 434
current time 434-----: SLO_no_migration:17, SLO_RL:3, 
current time 434-----: SLO_no_sum:9673, SLO_sum:2484, 
current_time: 435
current time 435-----: SLO_no_migration:12, SLO_RL:1, 
current time 435-----: SLO_no_sum:9685, SLO_sum:2485, 
current_time: 436
current time 436-----: SLO_no_migration:20, SLO_RL:2, 
current time 436-----: SLO_no_sum:9705, SLO_sum:2

current_time: 506
current time 506-----: SLO_no_migration:16, SLO_RL:6, 
current time 506-----: SLO_no_sum:10718, SLO_sum:2677, 
current_time: 507
current time 507-----: SLO_no_migration:14, SLO_RL:4, 
current time 507-----: SLO_no_sum:10732, SLO_sum:2681, 
current_time: 508
current time 508-----: SLO_no_migration:11, SLO_RL:1, 
current time 508-----: SLO_no_sum:10743, SLO_sum:2682, 
current_time: 509
current time 509-----: SLO_no_migration:9, SLO_RL:1, 
current time 509-----: SLO_no_sum:10752, SLO_sum:2683, 
current_time: 510
current time 510-----: SLO_no_migration:11, SLO_RL:3, 
current time 510-----: SLO_no_sum:10763, SLO_sum:2686, 
current_time: 511
current time 511-----: SLO_no_migration:9, SLO_RL:3, 
current time 511-----: SLO_no_sum:10772, SLO_sum:2689, 
current_time: 512
current time 512-----: SLO_no_migration:10, SLO_RL:1, 
current time 512-----: SLO_no_sum:10782, SLO_sum:2690, 
current_time: 513
current time 513-----: SLO_no_migration:15, SLO_RL:4, 
current time 513-----: SLO

current time 595-----: SLO_no_migration:9, SLO_RL:2, 
current time 595-----: SLO_no_sum:11398, SLO_sum:2767, 
current_time: 596
current time 596-----: SLO_no_migration:8, SLO_RL:0, 
current time 596-----: SLO_no_sum:11406, SLO_sum:2767, 
current_time: 597
current time 597-----: SLO_no_migration:11, SLO_RL:6, 
current time 597-----: SLO_no_sum:11417, SLO_sum:2773, 
current_time: 598
current time 598-----: SLO_no_migration:7, SLO_RL:1, 
current time 598-----: SLO_no_sum:11424, SLO_sum:2774, 
current_time: 599
current time 599-----: SLO_no_migration:5, SLO_RL:1, 
current time 599-----: SLO_no_sum:11429, SLO_sum:2775, 
current_time: 600
current time 600-----: SLO_no_migration:6, SLO_RL:1, 
current time 600-----: SLO_no_sum:11435, SLO_sum:2776, 
current_time: 601
current time 601-----: SLO_no_migration:7, SLO_RL:1, 
current time 601-----: SLO_no_sum:11442, SLO_sum:2777, 
current_time: 602
current time 602-----: SLO_no_migration:6, SLO_RL:1, 
current time 602-----: SLO_no_sum:11448, SLO_sum:

current time 671-----: SLO_no_migration:10, SLO_RL:3, 
current time 671-----: SLO_no_sum:11915, SLO_sum:2847, 
current_time: 672
current time 672-----: SLO_no_migration:4, SLO_RL:1, 
current time 672-----: SLO_no_sum:11919, SLO_sum:2848, 
current_time: 673
current time 673-----: SLO_no_migration:6, SLO_RL:0, 
current time 673-----: SLO_no_sum:11925, SLO_sum:2848, 
current_time: 674
current time 674-----: SLO_no_migration:3, SLO_RL:0, 
current time 674-----: SLO_no_sum:11928, SLO_sum:2848, 
current_time: 675
current time 675-----: SLO_no_migration:7, SLO_RL:1, 
current time 675-----: SLO_no_sum:11935, SLO_sum:2849, 
current_time: 676
current time 676-----: SLO_no_migration:8, SLO_RL:0, 
current time 676-----: SLO_no_sum:11943, SLO_sum:2849, 
current_time: 677
current time 677-----: SLO_no_migration:10, SLO_RL:0, 
current time 677-----: SLO_no_sum:11953, SLO_sum:2849, 
current_time: 678
current time 678-----: SLO_no_migration:6, SLO_RL:0, 
current time 678-----: SLO_no_sum:11959, SLO_sum

In [1802]:
agent.model.save("RL_100.h5")